In [0]:
%pip install bokeh

Python interpreter will be restarted.
  Using cached bokeh-2.4.2-py3-none-any.whl (18.5 MB)
  Using cached PyYAML-6.0-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (701 kB)
  Using cached typing_extensions-4.2.0-py3-none-any.whl (24 kB)
Python interpreter will be restarted.


In [0]:
path='dbfs:/FileStore/tables/clinicaltrial_2021.csv'
year=path[-8:-4]

In [0]:
clinicaltrial_2021=sc.textFile(path)
clinicaltrial_2021.take(5)

Out[2]: ['Id|Sponsor|Status|Start|Completion|Type|Submission|Conditions|Interventions',
 'NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|']

In [0]:
header=clinicaltrial_2021.first()
clinicaltrial_2021=clinicaltrial_2021.filter(lambda x: x!=header)
clinicaltrial_2021.take(5)

Out[3]: ['NCT02758028|The University of Hong Kong|Recruiting|Aug 2005|Nov 2021|Interventional|Apr 2016||',
 'NCT02751957|Duke University|Completed|Jul 2016|Jul 2020|Interventional|Apr 2016|Autistic Disorder,Autism Spectrum Disorder|',
 'NCT02758483|Universidade Federal do Rio de Janeiro|Completed|Mar 2017|Jan 2018|Interventional|Apr 2016|Diabetes Mellitus|',
 'NCT02759848|Istanbul Medeniyet University|Completed|Jan 2012|Dec 2014|Observational|May 2016|Tuberculosis,Lung Diseases,Pulmonary Disease|',
 'NCT02758860|University of Roma La Sapienza|Active, not recruiting|Jun 2016|Sep 2020|Observational [Patient Registry]|Apr 2016|Diverticular Diseases,Diverticulum,Diverticulosis|']

In [0]:
mesh=sc.textFile("FileStore/tables/mesh.csv")

mesh=mesh.map(lambda x:(x.split(",")))\
    .map(lambda x:(x[0],x[1]))

mesh.take(10)

Out[4]: [('term', 'tree'),
 ('Calcimycin', 'D03.633.100.221.173'),
 ('A-23187', 'D03.633.100.221.173'),
 ('Temefos', 'D02.705.400.625.800'),
 ('Temefos', 'D02.705.539.345.800'),
 ('Temefos', 'D02.886.300.692.800'),
 ('Abate', 'D02.705.400.625.800'),
 ('Abate', 'D02.705.539.345.800'),
 ('Abate', 'D02.886.300.692.800'),
 ('Difos', 'D02.705.400.625.800')]

In [0]:
pharma=sc.textFile("FileStore/tables/pharma/pharma.csv")

pharma=pharma.map(lambda x:x.replace('"',""))\
    .map(lambda x:x.split(",")[1])\
    .map(lambda x:x.lstrip())\
    .map(lambda x:(x,1))

pharma.take(10)

Out[5]: [('Parent_Company', 1),
 ('Abbott Laboratories', 1),
 ('AbbVie', 1),
 ('AbbVie', 1),
 ('Inc.', 1),
 ('Johnson & Johnson', 1),
 ('Abbott Laboratories', 1),
 ('Inc.', 1),
 ('Johnson & Johnson', 1),
 ('a division of Ethicon Inc.', 1)]

In [0]:
# Task-1
clinicaltrial_2021.map(lambda x:x.split("|")[0]).distinct().count()

Out[6]: 387261

In [0]:
# Task-2
clinicaltrial_2021.map(lambda x:x.split("|")[5])\
    .map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(ascending=False)\
    .map(lambda x:(x[1],x[0])).collect()

Out[7]: [('Interventional', 301472),
 ('Observational', 77540),
 ('Observational [Patient Registry]', 8180),
 ('Expanded Access', 69)]

In [0]:
# Task-3
conditions=clinicaltrial_2021.map(lambda x:x.split("|")[7]) \
    .flatMap(lambda x:x.split(","))\
    .filter(lambda x: x!="")

conditions.map(lambda x:(x,1))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(ascending=False)\
    .map(lambda x: (x[1],x[0])).take(5)

Out[8]: [('Carcinoma', 13389),
 ('Diabetes Mellitus', 11080),
 ('Neoplasms', 9371),
 ('Breast Neoplasms', 8640),
 ('Syndrome', 8032)]

In [0]:
# Task-4
conditions.map(lambda x:(x,1))\
    .join(mesh)\
    .map(lambda x:(x[1][1][:3],x[1][0]))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(ascending=False)\
    .map(lambda x:(x[1],x[0])).take(5)

Out[9]: [('C04', 143994),
 ('C23', 136079),
 ('C01', 106674),
 ('C14', 94523),
 ('C10', 92310)]

In [0]:
# Task-5
sponsor=clinicaltrial_2021.map(lambda x:x.split("|")[1])\
    .map(lambda x:(x,1))

sponsor.leftOuterJoin(pharma)\
    .filter(lambda x:(x[1][1]==None))\
    .map(lambda x:(x[0],x[1][0]))\
    .reduceByKey(lambda x,y:x+y)\
    .map(lambda x:(x[1],x[0]))\
    .sortByKey(ascending=False)\
    .map(lambda x:(x[1],x[0])).take(10)

Out[10]: [('National Cancer Institute (NCI)', 3218),
 ('M.D. Anderson Cancer Center', 2414),
 ('Assistance Publique - Hôpitaux de Paris', 2369),
 ('Mayo Clinic', 2300),
 ('Merck Sharp & Dohme Corp.', 2243),
 ('Assiut University', 2154),
 ('Novartis Pharmaceuticals', 2088),
 ('Massachusetts General Hospital', 1971),
 ('Cairo University', 1928),
 ('Hoffmann-La Roche', 1828)]

In [0]:
# Task-6
submissions=clinicaltrial_2021.map(lambda x:x.split("|"))\
    .filter(lambda x:'Completed' in x)\
    .map(lambda x:(x[4]))\
    .filter(lambda x: year in x)\
    .map(lambda x:(x[:3],1))\
    .reduceByKey(lambda x,y:x+y)

import calendar
c={x:y for y,x in enumerate(calendar.month_abbr[1:],1)}
submissions=submissions.sortBy(lambda x: c.get(x[0]))
submissions.collect()

Out[11]: [('Jan', 1131),
 ('Feb', 934),
 ('Mar', 1227),
 ('Apr', 967),
 ('May', 984),
 ('Jun', 1094),
 ('Jul', 819),
 ('Aug', 700),
 ('Sep', 528),
 ('Oct', 187)]

In [0]:
# Task-6
graph_data=submissions.collectAsMap()

from bokeh.io import output_file, show
from bokeh.plotting import figure
from bokeh.embed import file_html
from bokeh.resources import CDN
from bokeh.models import HoverTool

p = figure(x_range=list(graph_data), height=250, title="Submissions in Year " + year,
           toolbar_location=None, tools="hover",tooltips=[("Month", "@x"), ("Submissions", "@top")])

p.vbar(x=list(graph_data.keys()), top=list(graph_data.values()), width=0.9)

p.xgrid.grid_line_color = None
p.y_range.start = 0

html=file_html(p,CDN)

displayHTML(html)

<!DOCTYPE html>
 
 
 
 
 
 Bokeh Application